## Live Demo

In [ ]:
from PIL import Image
from fall_prediction import Fall_prediction
import matplotlib.pyplot as plt
import sys
import os
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')

class DevNull:
    def write(self, msg): pass
    def flush(self): pass

sys.stderr = DevNull()      # Ignore Python warnings

import cv2
from PIL import Image
from fall_prediction import Fall_prediction

cap = cv2.VideoCapture(0)  # 0 = default webcam

frame_buffer = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert OpenCV BGR image to PIL Image
    pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame_buffer.append(pil_img)

    # Keep only the last 3 frames
    if len(frame_buffer) > 3:
        frame_buffer.pop(0)

    # Run prediction when we have 3 frames
    if len(frame_buffer) == 3:
        img1, img2, img3 = frame_buffer
        fig = plt.figure(figsize=(12,12))
        fig.add_subplot(1, 3, 1)
        plt.imshow(img1)
        plt.title("image at time t")
        plt.axis("off")

        fig.add_subplot(1, 3, 2)
        plt.imshow(img2)
        plt.title("image at time t+1")
        plt.axis("off")

        fig.add_subplot(1, 3, 3)
        plt.imshow(img3)
        plt.title("image at time t+2")
        plt.axis("off")

        plt.show()

        response = Fall_prediction(*frame_buffer)
        if response:
            print("There is", response['category'])
            print("Confidence :", response['confidence'])
            print("Angle :", response['angle'])
            print("Keypoint_corr :", response['keypoint_corr'])
        else:
            print("There is no fall detection...")

    # Show the frame
    # cv2.imshow('Real-Time Fall Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()